## Assignment A1b: Functions and Computation

In [5]:
# Appropriate Libraries:
import math

### 1. Common functions

### 1a. `sinewave`

In [4]:
def sinewave(t, f=1.0, d=0.0):
    assert not f == 0
    phi = 360 * d * f
    return math.sin(2 * math.pi * f * t + phi)

### 1b. `gabor`

### 1c. `gammatone`

### 2. Simple computation

### 2a. `localmaxima`|

### 2b. `crossings`


### 2c. `envelope`